In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import time
# Setting pandas to display columns
pd.set_option('display.max_columns', None)

In [2]:
nfl_small2 = pd.read_csv('nfl_small_cleaned_plus_weather.csv',index_col='index')\
                                    .drop(columns=['Unnamed: 0','Unnamed: 0_x'])\
                                    .sort_values(by=['game_id','play_id'])
nfl_small2.shape

(173100, 70)

In [3]:
nfl_small2.head(3)

,play_id,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
270409,36,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,35.0,1,0,NaN,0.0,NE 35,0,0,S.Gostkowski kicks 65 yards from NE 35 to end ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0
270410,51,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,80.0,1,0,1.0,0.0,PIT 20,10,18,(15:00) De.Williams right tackle to PIT 38 for...,run,18.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,NaN,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0
270411,72,2015091000,2015-09-10 00:00:00,14:21,861.0,1761.0,3561.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,62.0,1,0,1.0,0.0,PIT 38,10,31,(14:21) B.Roethlisberger pass short right to A...,pass,9.0,short,-4.0,13.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-10 20:40:00,NaN,0 days 00:01:57.000000000,2015-09-10 20:41:57,2015-09-10 20:41:57,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0


In [4]:
# Creating Improved Game Name Column for Dashboard
nfl_small2['game_name'] = nfl_small2.apply(lambda x: str(x['away_team']) +\
                                           ' @ ' + str(x['home_team']) +\
                                           ' ' +str(x['game_id'])[:4] +\
                                           '-' + str(x['game_id'])[4:6] +\
                                            '-' + str(x['game_id'])[6:8],axis=1)

In [5]:
# Sorting all values to create some "end of" flags below
nfl_small2 = nfl_small2.sort_values(by=['index'],ascending=True)

In [6]:
# Setting flag for end of drive
nfl_small2['end_of_drive'] = 0

end_of_drive_indexes = list(nfl_small2.reset_index().groupby(['game_id','drive']).last()['index'].values)

nfl_small2.end_of_drive.loc[end_of_drive_indexes] = 1

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# Setting flag for end of game
nfl_small2['end_of_game'] = 0

end_of_game_indexes = list(nfl_small2.reset_index().groupby(['game_id']).last()['index'].values)
end_of_drive_indexes[:5]

nfl_small2.end_of_game.loc[end_of_game_indexes] = 1

In [8]:
# Setting flag for INTs
nfl_small2['INT'] = 0

int_indexes = list(nfl_small2[(nfl_small2['end_of_drive']==1)&\
                       (nfl_small2['fumble']!=1)&\
                        (nfl_small2['play_type'] =='pass')].reset_index()['index'].values)
                   
nfl_small2.INT.loc[int_indexes] = 1

In [9]:
# Getting teams to add team matrix for easier filtering
teams = list(nfl_small2.groupby('posteam').sum().index)
teams[:5]

['ARI', 'ATL', 'BAL', 'BUF', 'CAR']

In [10]:
# Adding team columns
for col in teams:
    nfl_small2[col] = '0'

In [11]:
# Setting all home teams
for team in teams:
    nfl_small2[team] = np.where((nfl_small2['home_team']==team),'H',nfl_small2[team])
    
# Setting all away teams
for team in teams:
    nfl_small2[team] = np.where((nfl_small2['away_team']==team),'A',nfl_small2[team])

In [12]:
nfl_small2['year'] = nfl_small2['year'].fillna(method='pad').astype('str')

In [13]:
# Fixing 2018, since it fails to join
nfl_small2['year'] = np.where((nfl_small2['game_date'] >= '2018-04-01'),'2018.0',nfl_small2['year'])

In [14]:
# Creating column for week of season
nfl_small2['week'] = None

# Getting every team by weeks played
for team in teams:
    week = nfl_small2[nfl_small2[team] != '0'][['year','game_date']]\
                .drop_duplicates().groupby('year').cumcount()+1
    
    nfl_small2.loc[list(week.index),'week'] = week.values
    
# Padding remaining plays per game with week of season    
nfl_small2['week'] = nfl_small2['week'].fillna(method='pad')

In [15]:
nfl_small2[nfl_small2['game_date']>='2018-01-01'].head()

,play_id,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
407335,37,2018090600,2018-09-06 00:00:00,15:00:00,900.0,1800.0,3600.0,Half1,0,1,PHI,ATL,ATL,away,PHI,PHI,35.0,1,0,NaN,0.0,PHI 35,0,73,J.Elliott kicks 65 yards from PHI 35 to end zo...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-09-06 16:25:00,2018.0,0 days 00:00:00.000000000,2018-09-06 16:25:00,2018-09-06 16:25:00,2018-09-06 16:00:00,1017.9,Philadelphia,21.1,Desso GrassMaster,46.0,0.0,Open,PHI,34.4,2018-09-06 16:00:00,2018-09-06 20:00:00,NaN,220.0,20.5,0,ATL @ PHI 2018-09-06,0,0,0,0,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,0,1
407336,52,2018090600,2018-09-06 00:00:00,15:00:00,900.0,1800.0,3600.0,Half1,0,1,PHI,ATL,ATL,away,PHI,ATL,75.0,1,0,1.0,0.0,ATL 25,10,73,"(15:00) PENALTY on ATL-L.Paulsen, False Start,...",no_play,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-09-06 16:25:00,2018.0,0 days 00:00:00.000000000,2018-09-06 16:25:00,2018-09-06 16:25:00,2018-09-06 16:00:00,1017.9,Philadelphia,21.1,Desso GrassMaster,46.0,0.0,Open,PHI,34.4,2018-09-06 16:00:00,2018-09-06 20:00:00,NaN,220.0,20.5,0,ATL @ PHI 2018-09-06,0,0,0,0,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,0,1
407337,75,2018090600,2018-09-06 00:00:00,15:00:00,900.0,1800.0,3600.0,Half1,0,1,PHI,ATL,ATL,away,PHI,ATL,80.0,1,0,1.0,0.0,ATL 20,15,73,(15:00) M.Ryan pass short right to J.Jones pus...,pass,10.0,short,8.0,2.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-09-06 16:25:00,2018.0,0 days 00:00:00.000000000,2018-09-06 16:25:00,2018-09-06 16:25:00,2018-09-06 16:00:00,1017.9,Philadelphia,21.1,Desso GrassMaster,46.0,0.0,Open,PHI,34.4,2018-09-06 16:00:00,2018-09-06 20:00:00,NaN,220.0,20.5,0,ATL @ PHI 2018-09-06,0,0,0,0,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,0,1
407338,104,2018090600,2018-09-06 00:00:00,14:22:00,862.0,1762.0,3562.0,Half1,0,1,PHI,ATL,ATL,away,PHI,ATL,70.0,1,0,2.0,0.0,ATL 30,5,73,(14:22) J.Jones left end pushed ob at ATL 41 f...,run,11.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-09-06 16:25:00,2018.0,0 days 00:01:54.000000000,2018-09-06 16:26:54,2018-09-06 16:26:54,2018-09-06 16:00:00,1017.9,Philadelphia,21.1,Desso GrassMaster,46.0,0.0,Open,PHI,34.4,2018-09-06 16:00:00,2018-09-06 20:00:00,NaN,220.0,20.5,0,ATL @ PHI 2018-09-06,0,0,0,0,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,0,1
407339,125,2018090600,2018-09-06 00:00:00,13:46:00,826.0,1726.0,3526.0,Half1,0,1,PHI,ATL,ATL,away,PHI,ATL,59.0,1,0,1.0,0.0,ATL 41,10,73,(13:46) D.Freeman right end to PHI 39 for 20 y...,run,20.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-09-06 16:25:00,2018.0,0 days 00:03:42.000000000,2018-09-06 16:28:42,2018-09-06 16:28:42,2018-09-06 16:00:00,1017.9,Philadelphia,21.1

In [16]:
# Chargers only appear to have two years in the data?
nfl_small2[(nfl_small2['posteam']=='LAC')&(nfl_small2['game_date']<= '2016-12-31')]

,play_id,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
272021,247,2015091308,2015-09-13 00:00:00,10:46,646.0,1546.0,3346.0,Half1,0,1,LAC,DET,LAC,home,DET,DET,35.0,2,0,NaN,0.0,DET 35,0,0,S.Martin kicks 70 yards from DET 35 to SD -5. ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-13 16:05:00,2015.0,0 days 00:12:42.000000000,2015-09-13 16:17:42,2015-09-13 16:17:42,2015-09-13 16:00:00,1011.2,San Diego,17.2,Grass,51.0,0.0,Open,LAC,28.3,2015-09-13 16:00:00,2015-09-13 20:00:00,6140.0,260.0,11.2,0,DET @ LAC 2015-09-13,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
272022,269,2015091308,2015-09-13 00:00:00,10:41,641.0,1541.0,3341.0,Half1,0,1,LAC,DET,LAC,home,DET,LAC,81.0,2,0,1.0,0.0,SD 19,10,5,(10:41) (Shotgun) P.Rivers pass short left to ...,pass,5.0,short,5.0,0.0,NaN,NaN,NaN,NaN,NaN,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-13 16:05:00,2015.0,0 days 00:12:57.000000000,2015-09-13 16:17:57,2015-09-13 16:17:57,2015-09-13 16:00:00,1011.2,San Diego,17.2,Grass,51.0,0.0,Open,LAC,28.3,2015-09-13 16:00:00,2015-09-13 20:00:00,6140.0,260.0,11.2,0,DET @ LAC 2015-09-13,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
272023,293,2015091308,2015-09-13 00:00:00,10:06,606.0,1506.0,3306.0,Half1,0,1,LAC,DET,LAC,home,DET,LAC,76.0,2,0,2.0,0.0,SD 24,5,3,(10:06) M.Gordon up the middle to SD 22 for -2...,run,-2.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-13 16:05:00,2015.0,0 days 00:14:42.000000000,2015-09-13 16:19:42,2015-09-13 16:19:42,2015-09-13 16:00:00,1011.2,San Diego,17.2,Grass,51.0,0.0,Open,LAC,28.3,2015-09-13 16:00:00,2015-09-13 20:00:00,6140.0,260.0,11.2,0,DET @ LAC 2015-09-13,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
272024,314,2015091308,2015-09-13 00:00:00,09:23,563.0,1463.0,3263.0,Half1,0,1,LAC,DET,LAC,home,DET,LAC,78.0,2,0,3.0,0.0,SD 22,7,-4,(9:23) (Shotgun) P.Rivers sacked at SD 15 for ...,pass,-7.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-13 16:05:00,2015.0,0 days 00:16:51.000000000,2015-09-13 16:21:51,2015-09-13 16:21:51,2015-09-13 16:00:00,1011.2,San Diego,17.2,Grass,51.0,0.0,Open,LAC,28.3,2015-09-13 16:00:00,2015-09-13 20:00:00,6140.0,260.0,11.2,0,DET @ LAC 2015-09-13,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
272025,333,2015091308,2015-09-13 00:00:00,08:42,522.0,1422.0,3222.0,Half1,0,1,LAC,DET,LAC,home,DET,LAC,85.0,2,0,4.0,0.0,SD 15,14,-4,"(8:42) M.Scifres punts 58 yards to DET 27, Cen...",punt,0.0,0,0.0,0.0,NaN,58.0,NaN,NaN,NaN,0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-13 16:05:00,2015.0,0 days 00:18:54.000000000,2015-09-13 16:23:54,2015-09-13 16:23:54,2015-09-13 16:00:00,1011.2,San Diego,17.2,Grass,51.0,0.0,Open,LAC,28.3,2015-09-13 16:00:00,2015-09-13 20:00:00,614

In [17]:
# Validating manipulation worked
nfl_small2[nfl_small2['LAC'] != '0'][['year','week']].drop_duplicates()

,year,week
index,,
272011,2015.0,1
274568,2015.0,2
277847,2015.0,3
280750,2015.0,4
284156,2015.0,5
286211,2015.0,6
290761,2015.0,8
294233,2015.0,9
298538,2015.0,10


In [111]:
nfl_small2['KC'] = nfl_small2['KC'].astype('str')
nfl_small2['LAC'] = nfl_small2['LAC'].astype('str')

In [112]:
# Saving nfl_small to csv to skip loading full file
nfl_small2.to_csv('nfl_small_end_of_drive.csv')